<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
#Import Even-Strength On-Ice Rates Data
skater_EV_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Rates.csv')
#skater_EV_rates_raw.head()

In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_rates = skater_EV_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','GF/60','GA/60','xGF/60','xGA/60']]
#skater_EV_rates

In [4]:
skater_EV_rates['EV_TOI'] = skater_EV_rates['TOI']*3600
#skater_EV_rates

In [5]:
#Convert rate/60 to rates/s
skater_EV_rates['GF_EV'] = skater_EV_rates['GF/60']/3600
skater_EV_rates['GA_EV'] = skater_EV_rates['GA/60']/3600
skater_EV_rates['xGF_EV'] = skater_EV_rates['xGF/60']/3600
skater_EV_rates['xGA_EV'] = skater_EV_rates['xGA/60']/3600
#skater_EV_rates

In [6]:
#Drop TOI, GF/60, GA/60, xGF/60 and xGA/60 columns
skater_EV_rates = skater_EV_rates.drop(columns = ['TOI','GF/60','GA/60','xGF/60','xGA/60'])
#skater_EV_rates

In [7]:
#Rename columns
skater_EV_rates = skater_EV_rates.rename(columns = {'Position':'Position_EV','GP':'GP_EV'})
#skater_EV_rates

In [8]:
#Import PP On-Ice Rates Data
skater_PP_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Rates.csv')
#skater_PP_rates_raw.head()

In [9]:
#Reduce PP On-Ice Totals Data
skater_PP_rates = skater_PP_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_PP_rates

In [10]:
#Convert G±/60 to G/s and TOI to PP_TOI(s)
skater_PP_rates['PP'] = skater_PP_rates['G±/60']/3600
skater_PP_rates['PP_TOI'] = skater_PP_rates['TOI']*3600
#skater_PP_rates

In [11]:
#Drop columns
skater_PP_rates = skater_PP_rates.drop(columns = ['TOI','G±/60'])
#skater_PP_rates

In [12]:
#Rename columns
skater_PP_rates = skater_PP_rates.rename(columns = {'Position':'Position_PP','GP':'GP_PP'})
#skater_PP_rates

In [13]:
#Import SH On-Ice Totals Data
skater_SH_rates_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Rates.csv')
#skater_SH_rates_raw.head()

In [14]:
#Reduce SH On-Ice Totals Data
skater_SH_rates = skater_SH_rates_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±/60']]
#skater_SH_rates

In [15]:
#Convert TOI to TOI(s) and G±/60 to G± (s)
skater_SH_rates['SH'] = skater_SH_rates['G±/60']/3600
skater_SH_rates['SH_TOI'] = skater_SH_rates['TOI']*3600
#skater_SH_rates

In [16]:
#Drop columns
skater_SH_rates = skater_SH_rates.drop(columns = ['G±/60','TOI'])
#skater_SH_rates

In [17]:
#Rename columns
skater_SH_rates = skater_SH_rates.rename(columns = {'Position':'Position_SH','GP':'GP_SH'})
#skater_SH_rates

In [18]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
#skater_box_totals_raw.head()

In [19]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2']]
#skater_box_totals.head()

In [20]:
skater_box_totals['TOI_tot'] = skater_box_totals['TOI']*3600
#skater_box_totals

In [21]:
skater_box_totals = skater_box_totals.drop(columns = {'TOI'})
#skater_box_totals

In [22]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine
#pd.options.mode.chained_assignment = None
#forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

#defense_table['PAA'] = defense_table['Points'] - Defense_mean
#defensemen_table.head(10)

#Add dataframes back together and sort again
#concat_frames = [forwards_table, defense_table]
#skater_box_totals = pd.concat(concat_frames)
#skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

In [23]:
#Combine skater EV and PP stats
skater_EV_PP = pd.merge(skater_EV_rates, skater_PP_rates, how="outer", on=['Player','Season'])
#skater_EV_PP

In [24]:
#Drop duplicate columns
skater_EV_PP = skater_EV_PP.drop(columns = ['Team_y','Position_PP','GP_PP'])
#skater_EV_PP

In [25]:
#Merge EV,PP,SH tables together
skater_EV_PP_SH = pd.merge(skater_EV_PP, skater_SH_rates, how="left", on=["Player", "Season"])
#skater_EV_PP_SH

In [26]:
#Clean up data
skater_EV_PP_SH = skater_EV_PP_SH.drop(columns = ['Team','Position_SH','GP_SH'])
#skater_EV_PP_SH

In [27]:
#Merge EV,PP,SH and Points tables together
full_skater_totals = pd.merge(skater_EV_PP_SH, skater_box_totals, how="left", on=["Player", "Season"])
#full_skater_totals.head()

In [28]:
#Replace NaN with 0
full_skater_totals = full_skater_totals.replace(np.NaN, 0)
#full_skater_totals

In [29]:
#Drop Repeated Columns
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position','GP'])
#full_skater_totals.head(10)


In [30]:
full_skater_totals = full_skater_totals.rename(columns = {'Team_x':'Team','Position_EV':'Position'})
#full_skater_totals

In [31]:
#Look at positions to get correct F/D average points
forwards_table = full_skater_totals.loc[full_skater_totals.Position!= "D"]
defense_table = full_skater_totals.loc[full_skater_totals.Position== "D"]

In [32]:
#Calculate Mean Stats of forwards and defensemen
avg_GF_EV_F = forwards_table['GF_EV'].mean()
avg_GA_EV_F = forwards_table['GA_EV'].mean()
avg_xGF_EV_F = forwards_table['xGF_EV'].mean()
avg_xGA_EV_F = forwards_table['xGA_EV'].mean()
avg_PP_F = forwards_table['PP'].mean()
avg_SH_F = forwards_table['SH'].mean()
avg_G_F = forwards_table['G'].mean()
avg_A1_F = forwards_table['A1'].mean()
avg_A2_F = forwards_table['A2'].mean()
#print(Forwards_mean)
avg_GF_EV_D = defense_table['GF_EV'].mean()
avg_GA_EV_D = defense_table['GA_EV'].mean()
avg_xGF_EV_D = defense_table['xGF_EV'].mean()
avg_xGA_EV_D = defense_table['xGA_EV'].mean()
avg_PP_D = defense_table['PP'].mean()
avg_SH_D = defense_table['SH'].mean()
avg_G_D = defense_table['G'].mean()
avg_A1_D = defense_table['A1'].mean()
avg_A2_D = defense_table['A2'].mean()
#print(Defensemen_mean)

In [33]:
#Add impact for all variables to the tables
pd.options.mode.chained_assignment = None
#forwards
forwards_table['GF_Impact'] = (forwards_table['GF_EV']-avg_GF_EV_F)*forwards_table['EV_TOI']
forwards_table['GA_Impact'] = (forwards_table['GA_EV']-avg_GA_EV_F)*forwards_table['EV_TOI']
forwards_table['xGF_Impact'] = (forwards_table['xGF_EV']-avg_xGF_EV_F)*forwards_table['EV_TOI']
forwards_table['xGA_Impact'] = (forwards_table['xGA_EV']-avg_xGA_EV_F)*forwards_table['EV_TOI']
forwards_table['PP_Impact'] = (forwards_table['PP']-avg_PP_F)*forwards_table['PP_TOI']
forwards_table['SH_Impact'] = (forwards_table['SH']-avg_SH_F)*forwards_table['SH_TOI']
forwards_table['G_Impact'] = (forwards_table['G']-avg_G_F)*forwards_table['TOI_tot']
forwards_table['A1_Impact'] = (forwards_table['A1']-avg_A1_F)*forwards_table['TOI_tot']
forwards_table['A2_Impact'] = (forwards_table['A2']-avg_A2_F)*forwards_table['TOI_tot']
#defense
defense_table['GF_Impact'] = (defense_table['GF_EV']-avg_GF_EV_D)*defense_table['EV_TOI']
defense_table['GA_Impact'] = (defense_table['GA_EV']-avg_GA_EV_D)*defense_table['EV_TOI']
defense_table['xGF_Impact'] = (defense_table['xGF_EV']-avg_xGF_EV_D)*defense_table['EV_TOI']
defense_table['xGA_Impact'] = (defense_table['xGA_EV']-avg_xGA_EV_D)*defense_table['EV_TOI']
defense_table['PP_Impact'] = (defense_table['PP']-avg_PP_D)*defense_table['PP_TOI']
defense_table['SH_Impact'] = (defense_table['SH']-avg_SH_D)*defense_table['SH_TOI']
defense_table['G_Impact'] = (defense_table['G']-avg_G_D)*defense_table['TOI_tot']
defense_table['A1_Impact'] = (defense_table['A1']-avg_A1_D)*defense_table['TOI_tot']
defense_table['A2_Impact'] = (defense_table['A2']-avg_A2_D)*defense_table['TOI_tot']

In [34]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
#goalie_stats_raw.head()

In [35]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','GSAx']]
#goalie_stats.head()

In [36]:
#Add Active / Inactive Column to both data tables
forwards_table['A/I'] = 'Active'
defense_table['A/I'] = 'Active'
goalie_stats['A/I'] = 'Active'

In [37]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
#active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
#json_api_data["teams"][0]

#player_name = json_api_data["teams"][0]
#player_team = 
#player_position = #code


#active_rosters.append{{"Player": player_name,
#                       "Team": player_team,
#                       "Postion": player_position}}




In [38]:
#Create Roster for each team
Avalanche_forwards = forwards_table.loc[(forwards_table.Player=="Andrew Cogliano")|(forwards_table.Player=="J.T. Compher")|(forwards_table.Player=="Darren Helm")|(forwards_table.Player=="Nazem Kadri")
|(forwards_table.Player=="Gabriel Landeskog")|(forwards_table.Player=="Artturi Lehkonen")|(forwards_table.Player=="Nathan MacKinnon")|(forwards_table.Player=="Alex Newhook")|(forwards_table.Player=="Valeri Nichushkin")
|(forwards_table.Player=="Logan O'Connor")|(forwards_table.Player=="Mikko Rantanen")|(forwards_table.Player=="Nico Sturm")]
Avalanche_defense = defense_table.loc[(defense_table.Player=="Bowen Byram")|(defense_table.Player=="Jack Johnson")
|(defense_table.Player=="Erik Johnson")|(defense_table.Player=="Cale Makar")|(defense_table.Player=="Josh Manson")|(defense_table.Player=="Devon Toews")]
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Player=="Darcy Kuemper")]
#Avalanche_forwards

In [39]:
#Create Avalanche roster for 2021-2022
Avalanche_forwards_2122 = Avalanche_forwards.loc[Avalanche_forwards.Season=="21-22"]
Avalanche_defense_2122 = Avalanche_defense.loc[Avalanche_defense.Season=="21-22"]
Avalanche_goalies_2122 = Avalanche_goalie.loc[Avalanche_goalie.Season=="21-22"]
#Avalanche_forwards_2122

In [40]:
#Create strength of roster for Avalanche
Avalanche_Strength_2122 = 0.5 + (Avalanche_forwards_2122['GF_Impact'].sum()*0.00294) + (Avalanche_forwards_2122['GA_Impact'].sum()*-0.00248) + (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Avalanche_forwards_2122['xGF_Impact'].sum()*0.00485) + (Avalanche_forwards_2122['PP_Impact'].sum()*0.00361) + (Avalanche_forwards_2122['SH_Impact'].sum()*0.00498) + (Avalanche_forwards_2122['G_Impact'].sum()*0.00749)
+ (Avalanche_forwards_2122['A1_Impact'].sum()*0.00771) + (Avalanche_forwards_2122['A2_Impact'].sum()*0.0102) + (Avalanche_defense_2122['GF_Impact'].sum()*0.00444) + (Avalanche_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Avalanche_defense_2122['xGF_Impact']*0.00736) + (Avalanche_defense_2122['xGA_Impact']*-0.00654) + (Avalanche_defense_2122['PP_Impact']*0.00833) + (Avalanche_defense_2122['SH_Impact']*0.00498)
+ (Avalanche_defense_2122['G_Impact'].sum()*0.0132) + (Avalanche_defense_2122['A1_Impact'].sum()*0.00963) + (Avalanche_defense_2122['A2_Impact'].sum()*0.00918) + (Avalanche_goalies_2122['GSAx']*0.00622)
Avalanche_Strength_2122

41.364297984805496

In [41]:
#Create Roster for Lightning
Lightning_forwards = forwards_table.loc[(forwards_table.Player=="Pierre-Edouard Bellemare")|(forwards_table.Player=="Anthony Cirelli")|(forwards_table.Player=="Ross Colton")|(forwards_table.Player=="Brandon Hagel")
|(forwards_table.Player=="Alex Killorn")|(forwards_table.Player=="Nikita Kucherov")|(forwards_table.Player=="Patrick Maroon")|(forwards_table.Player=="Riley Nash")|(forwards_table.Player=="Ondrej Palat")
|(forwards_table.Player=="Nick Paul")|(forwards_table.Player=="Corey Perry")|(forwards_table.Player=="Steven Stamkos")]
Lightning_defense = defense_table.loc[(defense_table.Player=="Zach Bogosian")|(defense_table.Player=="Erik Cernak")
|(defense_table.Player=="Victor Hedman")|(defense_table.Player=="Ryan McDonaugh")|(defense_table.Player=="Jan Rutta")|(defense_table.Player=="Mikhail Sergachev")]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Player=="Andrei Vasilevskiy")]
#Lightning_forwards

In [42]:
#Create Lightning roster for 2021-2022
Lightning_forwards_2122 = Avalanche_forwards.loc[Avalanche_forwards.Season=="21-22"]
Lightning_defense_2122 = Avalanche_defense.loc[Avalanche_defense.Season=="21-22"]
Lightning_goalies_2122 = Avalanche_goalie.loc[Avalanche_goalie.Season=="21-22"]
#Lightning_forwards_2122

In [43]:
#Create strength of Lightning roster for 2021-2022 season.
Lightning_Strength_2122 = 0.5 + (Lightning_forwards_2122['GF_Impact'].sum()*0.00294) + (Lightning_forwards_2122['GA_Impact'].sum()*-0.00248) + (Lightning_forwards_2122['xGF_Impact'].sum()*0.00485) + (Lightning_forwards_2122['xGA_Impact'].sum()*-0.00439)
+ (Lightning_forwards_2122['xGF_Impact'].sum()*0.00485) + (Lightning_forwards_2122['PP_Impact'].sum()*0.00361) + (Lightning_forwards_2122['SH_Impact'].sum()*0.00498) + (Lightning_forwards_2122['G_Impact'].sum()*0.00749)
+ (Lightning_forwards_2122['A1_Impact'].sum()*0.00771) + (Lightning_forwards_2122['A2_Impact'].sum()*0.0102) + (Lightning_defense_2122['GF_Impact'].sum()*0.00444) + (Lightning_defense_2122['GA_Impact'].sum()*-0.00385)
+ (Lightning_defense_2122['xGF_Impact']*0.00736) + (Lightning_defense_2122['xGA_Impact']*-0.00654) + (Lightning_defense_2122['PP_Impact']*0.00833) + (Lightning_defense_2122['SH_Impact']*0.00498)
+ (Lightning_defense_2122['G_Impact'].sum()*0.0132) + (Lightning_defense_2122['A1_Impact'].sum()*0.00963) + (Lightning_defense_2122['A2_Impact'].sum()*0.00918) + (Lightning_goalies_2122['GSAx']*0.00622)
Lightning_Strength_2122

41.364297984805496

In [44]:
#Game Projection for Game 6 of Stanley Cup Final
Lightning_Probability = Lightning_Strength_2122/(Lightning_Strength_2122+Avalanche_Strength_2122)
Avalanche_Probability = 1-Lightning_Probability
Lightning_Odds = ((1/Lightning_Probability)-1)*100
Avalanche_Odds = ((1/Avalanche_Probability)-1)*100

In [45]:
print("Lightning Odds:", Lightning_Odds)
print("Avalanche_Odds:", Avalanche_Odds)


Lightning Odds: 100.0
Avalanche_Odds: 100.0


In [ ]:
#Make a Data Table for skaters on each team
Avalanche_skaters = skater_stats.loc[(skater_stats.Team == 'COL')]
Blackhawks_skaters = skater_stats.loc[(skater_stats.Team == 'CHI')]
Blue_Jackets_skaters = skater_stats.loc[(skater_stats.Team == 'CBJ')]
Blues_skaters = skater_stats.loc[(skater_stats.Team == 'STL')]
Bruins_skaters = skater_stats.loc[(skater_stats.Team == 'BOS')]
Canadiens_skaters = skater_stats.loc[(skater_stats.Team == 'MTL')]
Canucks_skaters = skater_stats.loc[(skater_stats.Team == 'VAN')]
Capitals_skaters = skater_stats.loc[(skater_stats.Team == 'WSH')]
Coyotes_skaters = skater_stats.loc[(skater_stats.Team == 'ARI')]
Devils_skaters = skater_stats.loc[(skater_stats.Team == 'N.J')]
Ducks_skaters = skater_stats.loc[(skater_stats.Team == 'ANA')]
Flames_skaters = skater_stats.loc[(skater_stats.Team == 'CGY')]
Flyers_skaters = skater_stats.loc[(skater_stats.Team == 'PHI')]
Hurricanes_skaters = skater_stats.loc[(skater_stats.Team == 'CAR')]
Islanders_skaters = skater_stats.loc[(skater_stats.Team == 'NYI')]
Jets_skaters = skater_stats.loc[(skater_stats.Team == 'WPG')]
Kings_skaters = skater_stats.loc[(skater_stats.Team == 'L.A')]
Knights_skaters = skater_stats.loc[(skater_stats.Team == 'VGK')]
Kraken_skaters = skater_stats.loc[(skater_stats.Team == 'SEA')]
Leafs_skaters = skater_stats.loc[(skater_stats.Team == 'TOR')]
Lightning_skaters = skater_stats.loc[(skater_stats.Team == 'T.B')]
Oilers_skaters = skater_stats.loc[(skater_stats.Team == 'EDM')]
Panthers_skaters = skater_stats.loc[(skater_stats.Team == 'FLA')]
Penguins_skaters = skater_stats.loc[(skater_stats.Team == 'PIT')]
Predators_skaters = skater_stats.loc[(skater_stats.Team == 'NSH')]
Rangers_skaters = skater_stats.loc[(skater_stats.Team == 'NYR')]
Red_Wings_skaters = skater_stats.loc[(skater_stats.Team == 'DET')]
Sabres_skaters = skater_stats.loc[(skater_stats.Team == 'BUF')]
Senators_skaters = skater_stats.loc[(skater_stats.Team == 'OTT')]
Sharks_skaters = skater_stats.loc[(skater_stats.Team == 'S.J')]
Stars_skaters = skater_stats.loc[(skater_stats.Team == 'DAL')]
Wild_skaters = skater_stats.loc[(skater_stats.Team == 'MIN')]

In [ ]:
#Make a Data for each team of goalies
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Team == 'COL')]
Blackhawks_goalie = goalie_stats.loc[(goalie_stats.Team == 'CHI')]
Blue_Jackets_goalie = goalie_stats.loc[(goalie_stats.Team == 'CBJ')]
Blues_goalie = goalie_stats.loc[(goalie_stats.Team == 'STL')]
Bruins_goalie = goalie_stats.loc[(goalie_stats.Team == 'BOS')]
Canadiens_goalie = goalie_stats.loc[(goalie_stats.Team == 'MTL')]
Canucks_goalie = goalie_stats.loc[(goalie_stats.Team == 'VAN')]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Team == 'WSH')]
Coyotes_goalie = goalie_stats.loc[(goalie_stats.Team == 'ARI')]
Devils_goalie = goalie_stats.loc[(goalie_stats.Team == 'N.J')]
Ducks_goalie = goalie_stats.loc[(goalie_stats.Team == 'ANA')]
Flames_goalie = goalie_stats.loc[(goalie_stats.Team == 'CGY')]
Flyers_goalie = goalie_stats.loc[(goalie_stats.Team == 'PHI')]
Hurricanes_goalie = goalie_stats.loc[(goalie_stats.Team == 'CAR')]
Islanders_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYI')]
Jets_goalie = goalie_stats.loc[(goalie_stats.Team == 'WPG')]
Kings_goalie = goalie_stats.loc[(goalie_stats.Team == 'L.A')]
Knights_goalie = goalie_stats.loc[(goalie_stats.Team == 'VGK')]
Kraken_goalie = goalie_stats.loc[(goalie_stats.Team == 'SEA')]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Team == 'TOR')]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Team == 'T.B')]
Oilers_goalie = goalie_stats.loc[(goalie_stats.Team == 'EDM')]
Panthers_goalie = goalie_stats.loc[(goalie_stats.Team == 'FLA')]
Penguins_goalie = goalie_stats.loc[(goalie_stats.Team == 'PIT')]
Predators_goalie = goalie_stats.loc[(goalie_stats.Team == 'NSH')]
Rangers_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYR')]
Red_Wings_goalie = goalie_stats.loc[(goalie_stats.Team == 'DET')]
Sabres_goalie = goalie_stats.loc[(goalie_stats.Team == 'BUF')]
Senators_goalie = goalie_stats.loc[(goalie_stats.Team == 'OTT')]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Team == 'S.J')]
Stars_goalie = goalie_stats.loc[(goalie_stats.Team == 'DAL')]
Wild_goalie = goalie_stats.loc[(goalie_stats.Team == 'MIN')]